# Optimisation Target Analysis

In [1]:
from cobra.io import read_sbml_model
from cameo.strain_design import OptGene, OptKnock

In [2]:
# Get the model for analysis
model = read_sbml_model('data/iYO844_modified.xml')
model.solver = "glpk" # mathematical programming solver
model

Name,iYO844_modified
Memory address,0x01a77178aa08
Number of metabolites,996
Number of reactions,1258
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


## Optimisation target analysis using OptGene

In [3]:
# Run analysis with OptGene to find optimisation targets
optgene = OptGene(model)
result = optgene.run(target=model.reactions.ALDH1_CYP71AV1,     # Reaction producing dihydroartemisinic acid
                     biomass=model.reactions.BIOMASS_BS_10,
                     substrate=model.reactions.EX_glc__D_e,     # Using glucose as substrate
                     max_evaluations=5000,
                     plot=False)

result

Starting optimization at Tue, 05 Dec 2023 11:39:54


HBox()

No solutions found


Finished after 00:04:42


,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


OptGene has not found any options for optimising the production of dihydroartemisinic acid.

## Knock-out target analysis with OptKnock

### With the full GSM

In [ ]:
# Analysis with OptKnock of the reactions not involved with biomass or dihydroartemisinic acid production
optknock = OptKnock(model, fraction_of_optimum=0.1)
result = optknock.run(max_knockouts=1,
                      target="ALDH1_CYP71AV1",     # Production of dihydroartemisinic acid
                      biomass="BIOMASS_BS_10")

result

This code takes a long time because all the genes are tried for being knocked out. Even essensial ones. Hence it is necessary to narrow the list of potential target genes down.

### With non-essential genes as possible knockouts
Because of the necessary time for OptKnock to run, the possible knock-out targets are narrowed down from all reactions in the cell to all the reactions not necessary for either biomass formation or product production.

According to the documentation for OptKnock, inserting a list of reactions after the model will make these reactions the possible knockout targets. This documentation can be found at https://opencobra.github.io/cobratoolbox/stable/modules/design/index.html?highlight=OptKnock#src.design..OptKnock.

In [4]:
# Finding non-essential reactions for OptKnock to improve runtime
non_essential = []
not_necessary = []

# Looping through all reactions in the model and knocking them out one at a time
for reaction in model.reactions:
    with model:
        reaction.knock_out()
        # The ability to produce biomass with the knock-out leads to the reaction being deemed non-essential
        # Essential reactions are marked with 0. as the model objective can't be reached and are filtered out
        if model.slim_optimize(error_value=0.) != 0.0:
            non_essential.append(reaction)

# Looping through all non-essential reactions and knocking them out one at a time
for reaction in non_essential:
    with model:
        reaction.knock_out()
        # Model objective is formation of dihydroartemisinic acid
        model.objective = model.reactions.ALDH1_CYP71AV1
        # The ability to produce the product with the knock-out leads to the reaction being deemed not important
        if model.slim_optimize(error_value=0.) != 0.0:
            not_necessary.append(reaction)

In [5]:
# Check that the not_necessary reactions is the smallest list
print(len(model.reactions),len(non_essential),len(not_necessary))

1258 1191 1186


In [ ]:
# Analysis with OptKnock of the reactions not involved with biomass or dihydroartemisinic acid production
optknock = OptKnock(model, not_necessary, fraction_of_optimum=0.1)
result = optknock.run(max_knockouts=1,
                      target="ALDH1_CYP71AV1",     # Production of dihydroartemisinic acid
                      biomass="BIOMASS_BS_10")

result

### Excluding essential genes

Because of the necessary time for OptKnock to run, the possible knock-out targets are narrowed down from all reactions in the cell to all the reactions not necessary for either biomass formation or product production.

According to the documentation for the OptKnock function, when getting help to the function within the code (see picture below), the function will always exclude essensial reactions from the list of genes to knock-out.
What we want to do is to find the essential genes for production of dihydroartemisinic acid and also exclude these from the list of genes to knock-out.

![OptKnock documentation](figures/OptKnock_class_doc.png)

This is because the function has an option to include a list of genes to exclude from trying to knock them out.

In [6]:
# Finding essential reactions for OptKnock to exclude from the possible genes to knock out
# This will likely improve runtime
essential_DHAA = []

# Looping through all reactions in the model and knocking them out one at a time
for reaction in model.reactions:
    with model:
        reaction.knock_out()
        # Model objective is formation of dihydroartemisinic acid (DHAA)
        model.objective = model.reactions.ALDH1_CYP71AV1
        # The inability to produce DHAA with the knock-out leads to the reaction being essential
        # Essential reactions are marked with 0. as the model objective can't be reached and are collected
        if model.slim_optimize(error_value=0.) == 0.0:
                essential_DHAA.append(reaction)

# Check number of essential genes
print(len(essential_DHAA))

12


In [ ]:
# Analysis with OptKnock of the reactions not involved with biomass or dihydroartemisinic acid production
optknock = OptKnock(model, essential_DHAA, fraction_of_optimum=0.1)
result_optknock = optknock.run(max_knockouts=1,
                      target="ALDH1_CYP71AV1",     # Production of dihydroartemisinic acid
                      biomass="BIOMASS_BS_10")

print(result_optknock)